In [1]:
import os, sys
import csv
import platform
import numpy as np
import pandas as pd
import dask.dataframe as dd
import scipy
import sklearn
import matplotlib as plt
import flask
import math

In [2]:
path_data = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/11_CFX/05_PhD runs/R67_fluent/peak_R67_hgt_512k/trn_des_v01/data'
path_post = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/11_CFX/05_PhD runs/R67_fluent/peak_R67_hgt_512k/trn_des_v01/post' 
path_acu = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/11_CFX/05_PhD runs/R67_fluent/peak_R67_hgt_512k/trn_des_v01/post/acu'
path_plots = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/11_CFX/05_PhD runs/R67_fluent/peak_R67_hgt_512k/trn_des_v01/post/plots'

In [3]:
batch_data = dd.read_csv('data/asdp_data-*.dat', sep='\s+', decimal='.')
averages = pd.DataFrame(batch_data.groupby('nodenumber').mean().compute())

In [24]:
averages.columns

Index(['x-coordinate', 'y-coordinate', 'z-coordinate', 'pressure',
       'velocity-magnitude', 'sound-speed', 'temperature',
       'rel-velocity-magnitude', 'relative-velocity-angle'],
      dtype='object')

In [27]:
avg_static_p = pd.DataFrame({
    'x-coordinate': averages['x-coordinate'],
    'y-coordinate': averages['y-coordinate'],
    'z-coordinate': averages['z-coordinate'],
    'pressure': averages['pressure']
})

In [30]:
avg_static_p = avg_static_p[['x-coordinate', 'y-coordinate', 'z-coordinate', 'pressure']]
avg_static_p

,x-coordinate,y-coordinate,z-coordinate,pressure
nodenumber,,,,
1,-0.022169,0.103988,-0.043006,87558.887444
2,-0.021974,0.104226,-0.040024,87470.261845
3,-0.021503,0.104538,-0.037082,87366.194012
4,-0.020723,0.104936,-0.034216,87248.309902
5,-0.019646,0.105376,-0.031455,87124.131776
6,-0.018339,0.105883,-0.028802,86980.795716
7,-0.016864,0.106402,-0.026241,86806.682908
8,-0.015243,0.106910,-0.023769,86599.171029
9,-0.013530,0.107449,-0.021364,86363.092099


In [5]:
del(batch_data)

In [6]:
filelist = os.listdir(path_data)

In [7]:
for file in filelist:
    os.chdir(path_data)
    timestep = str(os.path.basename(str(file)))[10:-4]
    time_static_p = pd.DataFrame(pd.read_csv(file, sep='\s+', header=0, usecols=["nodenumber", "x-coordinate", "y-coordinate", "z-coordinate", "pressure"], skiprows=0, decimal='.')).set_index('nodenumber')
    acoustic_p = time_static_p.subtract(avg_static_p, fill_value=None)
    os.chdir(path_acu)
    acoustic_p.to_csv(str('asdp_acu_p_' + str(timestep) + '.dat'), sep=',')

In [8]:
os.chdir(path_acu)
filelist = os.listdir(path_acu)
filelist

['asdp_acu_p_0001.dat',
 'asdp_acu_p_0002.dat',
 'asdp_acu_p_0003.dat',
 'asdp_acu_p_0004.dat',
 'asdp_acu_p_0005.dat',
 'asdp_acu_p_0006.dat',
 'asdp_acu_p_0007.dat',
 'asdp_acu_p_0008.dat',
 'asdp_acu_p_0009.dat',
 'asdp_acu_p_0010.dat',
 'asdp_acu_p_0011.dat',
 'asdp_acu_p_0012.dat',
 'asdp_acu_p_0013.dat',
 'asdp_acu_p_0014.dat',
 'asdp_acu_p_0015.dat',
 'asdp_acu_p_0016.dat',
 'asdp_acu_p_0017.dat',
 'asdp_acu_p_0018.dat',
 'asdp_acu_p_0019.dat',
 'asdp_acu_p_0020.dat',
 'asdp_acu_p_0021.dat',
 'asdp_acu_p_0022.dat',
 'asdp_acu_p_0023.dat',
 'asdp_acu_p_0024.dat',
 'asdp_acu_p_0025.dat',
 'asdp_acu_p_0026.dat',
 'asdp_acu_p_0027.dat',
 'asdp_acu_p_0028.dat',
 'asdp_acu_p_0029.dat',
 'asdp_acu_p_0030.dat',
 'asdp_acu_p_0032.dat',
 'asdp_acu_p_0033.dat',
 'asdp_acu_p_0034.dat',
 'asdp_acu_p_0035.dat',
 'asdp_acu_p_0036.dat',
 'asdp_acu_p_0037.dat',
 'asdp_acu_p_0038.dat',
 'asdp_acu_p_0039.dat',
 'asdp_acu_p_0040.dat',
 'asdp_acu_p_0041.dat',
 'asdp_acu_p_0042.dat',
 'asdp_acu_p_004

In [31]:
os.chdir(path_data)
time_static_p = pd.DataFrame(pd.read_csv(file, sep='\s+', header=0, usecols=["nodenumber", "x-coordinate", "y-coordinate", "z-coordinate", "pressure"], skiprows=0, decimal='.')).set_index('nodenumber')
time_static_p

,x-coordinate,y-coordinate,z-coordinate,pressure
nodenumber,,,,
1,-0.022169,0.103988,-0.043006,87530.30147
2,-0.021974,0.104226,-0.040024,87447.52649
3,-0.021503,0.104538,-0.037082,87310.88525
4,-0.020723,0.104936,-0.034216,87203.36975
5,-0.019646,0.105376,-0.031455,87111.05493
6,-0.018339,0.105883,-0.028802,86997.51538
7,-0.016864,0.106402,-0.026241,86834.70215
8,-0.015243,0.106910,-0.023769,86663.98584
9,-0.013530,0.107449,-0.021364,86431.41370


In [32]:
avg_static_p

,x-coordinate,y-coordinate,z-coordinate,pressure
nodenumber,,,,
1,-0.022169,0.103988,-0.043006,87558.887444
2,-0.021974,0.104226,-0.040024,87470.261845
3,-0.021503,0.104538,-0.037082,87366.194012
4,-0.020723,0.104936,-0.034216,87248.309902
5,-0.019646,0.105376,-0.031455,87124.131776
6,-0.018339,0.105883,-0.028802,86980.795716
7,-0.016864,0.106402,-0.026241,86806.682908
8,-0.015243,0.106910,-0.023769,86599.171029
9,-0.013530,0.107449,-0.021364,86363.092099


In [33]:
acoustic_p = time_static_p.subtract(avg_static_p, fill_value=None)

In [34]:
acoustic_p

,x-coordinate,y-coordinate,z-coordinate,pressure
nodenumber,,,,
1,-3.469447e-18,-1.387779e-17,0.000000e+00,-28.585974
2,3.469447e-18,2.775558e-17,-6.938894e-18,-22.735355
3,0.000000e+00,0.000000e+00,6.938894e-18,-55.308762
4,3.469447e-18,2.775558e-17,0.000000e+00,-44.940152
5,-3.469447e-18,0.000000e+00,0.000000e+00,-13.076846
6,0.000000e+00,1.387779e-17,0.000000e+00,16.719664
7,3.469447e-18,0.000000e+00,3.469447e-18,28.019242
8,-1.734723e-18,1.387779e-17,0.000000e+00,64.814811
9,0.000000e+00,0.000000e+00,0.000000e+00,68.321601


squares = batch_data**2

squares

means = squares.groupby('nodenumber').mean().compute()

roots = np.sqrt(means)
roots